In [22]:
import pandas_datareader.data as web
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, resample_apply
import pandas as pd
import talib as ta
import importlib
import BacktestingStrategies.Optimization as opt
import BacktestingStrategies.Prod_Strategy_Rsi as btstRsi
import importlib
importlib.reload(btstRsi)

#https://www.sbisec.co.jp/ETGate/WPLETmgR001Control?OutSide=on&getFlg=on&burl=search_domestic&cat1=domestic&cat2=none&dir=info&file=domestic_info150818_01.html
#https://www.morningstar.co.jp/etf_foreign/search.jsp
# start = datetime.date(2021,1,1)
start = datetime.datetime.today() + relativedelta.relativedelta(years=-1)
end = datetime.date.today()
# data = web.DataReader('1570.JP', 'stooq', start, end)     #NEXT FUNDS 日経平均レバレッジ・インデックス連動型上場投信
# data = web.DataReader('1357.JP', 'stooq', start, end)     #NEXT FUNDS 日経平均ダブルインバース・インデックス連動型上場投信
# data = web.DataReader('1306.JP', 'stooq', start, end)     #NEXT FUNDS TOPIX連動型上場投信
# data = web.DataReader('1540.JP', 'stooq', start, end)     #純金上場信託(現物国内保管型)
# data = web.DataReader('2558.JP', 'stooq', start, end)     #MAXIS 米国株式(S&P500)上場投信
data = web.DataReader('2559.JP', 'stooq', start, end)      #ＭＡＸＩＳ全世界株式（オール・カントリー）上場投信
#data = web.DataReader('1655.JP', 'stooq', start, end)      #ｉシェアーズ　Ｓ＆Ｐ　５００　米国株　ＥＴＦ（1655）

data = data.astype("double")
print(data)

initial = 1000_000
fname = f"html/plot-"

#position
#0 ... display "only long" result
#1 ... display both "only long" and "long and short" result
position = 1

df_returns = pd.DataFrame()

               Open     High      Low    Close   Volume
Date                                                   
2022-11-02  14345.0  14365.0  14325.0  14360.0   2611.0
2022-11-01  14365.0  14445.0  14350.0  14445.0   6661.0
2022-10-31  14365.0  14395.0  14330.0  14345.0   9430.0
2022-10-28  14095.0  14155.0  14020.0  14025.0   6117.0
2022-10-27  14200.0  14230.0  14165.0  14230.0   3215.0
...             ...      ...      ...      ...      ...
2021-11-11  14060.0  14140.0  14060.0  14120.0   2816.0
2021-11-10  14070.0  14100.0  14010.0  14040.0   4015.0
2021-11-09  14220.0  14230.0  14040.0  14080.0  10344.0
2021-11-08  14160.0  14340.0  14130.0  14200.0  12565.0
2021-11-05  14370.0  14400.0  14150.0  14160.0  26023.0

[243 rows x 5 columns]


**Methods**

In [23]:
import datetime

def execute_backtest(bt):
    output = bt.run()
    print(output)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html") # 実行結果（グラフ）
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    global df_returns
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

    t_delta = datetime.timedelta(hours=9)
    JST = datetime.timezone(t_delta, 'JST')
    now = datetime.datetime.now(JST)
    print(repr(now))
    print(now)  # 2021-11-04 17:37:28.114417+09:00
    d = now.date().strftime('%Y%m%d')
    print(d)

    output._trades.to_csv('./Output/trades_' + d +  ".csv")

**RSI**

In [24]:
importlib.reload(opt)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstRsi.RsiOscillator, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

RSI close 2022-01-06 00:00:00
stop loss  2022-02-24 00:00:00  13697.4685  12756.435476064673
take profit  2022-04-05 00:00:00  12677.44425  14368.751259430417
RSI close 2022-06-07 00:00:00
RSI close 2022-07-25 00:00:00
RSI close 2022-11-02 00:00:00
Start                     2021-11-05 00:00:00
End                       2022-11-02 00:00:00
Duration                    362 days 00:00:00
Exposure Time [%]                   61.316872
Equity Final [$]                1243678.06725
Equity Peak [$]                 1252278.06725
Return [%]                          24.367807
Buy & Hold Return [%]                1.412429
Return (Ann.) [%]                   25.376366
Volatility (Ann.) [%]               19.402968
Sharpe Ratio                          1.30786
Sortino Ratio                        2.479259
Calmar Ratio                         2.941707
Max. Drawdown [%]                   -8.626408
Avg. Drawdown [%]                   -2.683422
Max. Drawdown Duration       75 days 00:00:00
Avg. Drawdown D

C:\Users\javie\AppData\Local\Temp\ipykernel_19820\619358908.py:4: UserWarning: Data index is not sorted in ascending order. Sorting.
  bt = Backtest(


,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,71,16,42,13978.85450,14150.0,12151.33050,0.012243,2021-11-30,2022-01-07,38 days
1,73,50,74,13697.46850,13090.0,-44345.20050,-0.044349,2022-01-20,2022-02-25,36 days
2,76,81,101,12677.44425,14475.0,136614.23700,0.141792,2022-03-08,2022-04-06,29 days
3,81,122,142,13601.99825,14220.0,50058.14175,0.045435,2022-05-11,2022-06-08,28 days
4,85,147,175,13466.33000,13825.0,30486.95000,0.026635,2022-06-15,2022-07-26,41 days
5,86,217,242,13662.29525,14345.0,58712.60850,0.049970,2022-09-27,2022-11-02,36 days


datetime.datetime(2022, 11, 5, 12, 50, 55, 165585, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400), 'JST'))
2022-11-05 12:50:55.165585+09:00
20221105
